In [1]:
'''
Version 1.0
Doesn't check the other branch, but works for now
Doesn't use custom minimize function 
Almost no comments

'''
import numpy as np
from math import floor
from scipy.optimize import minimize 

In [2]:
def f(x):
    x1 = x[0]
    x2 = x[1]

    return -(100*x1+150*x2)         ## Maximize f(x) == Minimiz -f(x)

In [3]:
def g1(x):
    x1 = x[0]
    x2 = x[1]

    return -(8000*x1+4000*x2-40000)      ## Minus needed for minimization 

In [4]:
def g2(x):
    x1 = x[0]
    x2 = x[1]

    return -(15*x1+30*x2-200  )          ## Minus needed for minimization 

In [5]:
def solution(fun, bounds, constraints):
    x_0 = [1,1]         ##inital guess
    return minimize(fun, x_0, method='SLSQP', bounds = bounds, constraints = constraints)


In [6]:
def add_bounds(bounds, b_start, b_end):
    bound = [b_start, b_end]
    bounds.append(bound)


In [7]:
def add_constraints(constraints, type, constraint_function):
    constraint = {'type': type, 'fun': constraint_function}
    constraints.append(constraint) 

In [8]:
def x_is_integer(x):
    rem = 0
    for i in x:
        rem += i%1
    return rem == 0

In [9]:
def get_largest_dec_index(x):
    index = 0
    temp = 0

    for i in range(len(x)):
        r = x[i]%1
        if(r > temp):
            temp = r
            index = i
    return index

In [10]:
def round_array(x):
    ret = []
    for i in range(len(x)):
        ret.append(round(x[i],2))
    return ret

In [11]:
def floor_array(x):
    ret = []
    for i in range(len(x)):
        ret.append(floor(x[i]))
    return ret

In [12]:
def BnB(fun, bounds, constraints):

    print(bounds)

    main_sol = solution(fun, bounds, constraints)
    x = round_array(main_sol.x)
    
    if(x_is_integer(x)):
        return {'x': x, 'solution': -fun(x)}
    UB = -fun(x)
    LB = -fun(floor_array(x))

    index = get_largest_dec_index(x)
    a = floor(x[index])
    b = a + 1

    left_bounds = bounds.copy()
    left_bounds[index] = [left_bounds[index][0], a]
    sol1 = solution(fun, left_bounds, constraints)
    res1 = -sol1.fun if (sol1.success or x_is_integer(sol1.x)) and (-sol1.fun <=UB and -sol1.fun >= LB) else -float('inf')

    right_bounds = bounds.copy()
    right_bounds[index] = [b, right_bounds[index][1]]
    sol2 = solution(fun, right_bounds, constraints)
    res2 = -sol2.fun if sol2.success or x_is_integer(sol2.x) and (-sol2.fun <=UB and -sol2.fun >= LB) else -float('inf')

    if(res1>res2):
        return BnB(fun,left_bounds,constraints)
    else:
        return BnB(fun,right_bounds,constraints)

In [13]:
bounds = []

add_bounds(bounds, 0, float('inf'))
add_bounds(bounds, 0, float('inf'))
'''
add_bounds(bounds, 0, 1)
add_bounds(bounds, 6, float('inf'))
'''
constraints = []
add_constraints(constraints, 'ineq', g1)
add_constraints(constraints, 'ineq', g2)

#print(x_is_integer(solution(f, bounds, constraints).x))

print(BnB(f,bounds, constraints))

[[0, inf], [0, inf]]
[2.22, 5.56]
1000.0000000490367 1033.3333336397714

[[0, inf], [6, inf]]
[1.33, 6.0]
1025.0000000033324 -inf

[[0, 1], [6, inf]]
[1.0, 6.17]
1000.0 -inf

[[0, 1], [6, 6]]
[1.0, 6.0]
{'x': [1.0, 6.0], 'solution': 1000.0}
C:\Users\Albert\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx


In [14]:
def f1(x):
    x1 = x[0]
    x2 = x[1]
    return -(17*x1+12*x2)

In [15]:
def g1_1(x):
    x1 = x[0]
    x2 = x[1]

    return -(10*x1+7*x2-40)

In [16]:
def g1_2(x):
    x1 = x[0]
    x2 = x[1]

    return -(x1+x2-5)


In [17]:
bounds1 = []
'''
add_bounds(bounds1, 0, float('inf'))
add_bounds(bounds1, 0, float('inf'))
'''
add_bounds(bounds1, 4, float('inf'))
add_bounds(bounds1, 0, 1)

constraints1 = []
add_constraints(constraints1, 'ineq', g1_1)
add_constraints(constraints1, 'ineq', g1_2)

#print(round_array(solution(f1, bounds1, constraints1).x))

print(BnB(f1,bounds1, constraints1))

[[4, inf], [0, 1]]
[4.0, 0.0]
{'x': [4.0, 0.0], 'solution': 68.0}
